In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightgbm as lgb
import seaborn as sns

In [39]:
df = pd.read_csv('/Users/estyle-150/github/my_signage/data/processed/processed20240826_2.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

train_data = df[df['is_train'] == 1].copy()
test_data = df[df['is_train'] == 0].copy()

train_data.drop(['is_train'], axis=1, inplace=True)
test_data.drop(['y','is_train'], axis=1, inplace=True)

display(train_data.head(), test_data.head())


,datetime,y,client,close,price_am,price_pm,price,y_lag_365,price_am_lag_1,price_am_lag_7,price_am_lag_365,price_pm_lag_1,price_pm_lag_7,price_pm_lag_365,price_am_moving_avg_7,price_am_moving_avg_30,price_pm_moving_avg_7,price_pm_moving_avg_30
0,2011-01-04,16,0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-05,16,0,0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-06,13,0,0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-01-07,14,0,0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-01-08,16,0,0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


,datetime,client,close,price_am,price_pm,price,y_lag_365,price_am_lag_1,price_am_lag_7,price_am_lag_365,price_pm_lag_1,price_pm_lag_7,price_pm_lag_365,price_am_moving_avg_7,price_am_moving_avg_30,price_pm_moving_avg_7,price_pm_moving_avg_30
1914,2016-04-01,1,0,3.0,2.0,2.5,60.0,5.0,4.0,2.0,4.0,5.0,2.0,4.428571,2.400000,3.857143,2.266667
1915,2016-04-02,0,0,5.0,5.0,5.0,75.0,3.0,5.0,2.0,2.0,5.0,1.0,4.428571,2.566667,3.857143,2.433333
1916,2016-04-03,1,0,2.0,2.0,2.0,86.0,5.0,5.0,4.0,5.0,4.0,3.0,4.000000,2.600000,3.571429,2.500000
1917,2016-04-04,1,0,1.0,1.0,1.0,69.0,2.0,4.0,2.0,2.0,4.0,2.0,3.571429,2.566667,3.142857,2.500000
1918,2016-04-05,0,0,1.0,1.0,1.0,61.0,1.0,4.0,1.0,1.0,4.0,1.0,3.142857,2.566667,2.714286,2.500000


In [40]:
from sklearn.model_selection import KFold
import optuna.integration.lightgbm as lgb_op
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val = train_test_split(train_data, test_size=0.2, shuffle=True, random_state=9)

col = 'y'
train_y = df_train[col]
train_x = df_train.drop(col,axis=1)

val_y = df_val[col]
val_x = df_val.drop(col,axis=1)

trains = lgb.Dataset(train_x,train_y)
valids = lgb.Dataset(val_x,val_y)

params = {
  'objective':'regression',
  'metric': 'mae',
  'force_row_wise' : True,
  'verbose': -1
}

tuner = lgb_op.LightGBMTunerCV(params, trains,num_boost_round=1000, optuna_seed=123, show_progress_bar=True)
tuner.run()

print('Best score:' ,{tuner.best_score})
print('Best params:')
print(tuner.best_params)


[I 2024-08-26 13:30:00,730] A new study created in memory with name: no-name-cbecf3ef-4ff7-4530-bdeb-3d7669a4365c
feature_fraction, val_score: inf:   0%|                                                                      | 0/7 [00:00<?, ?it/s][W 2024-08-26 13:30:00,802] Trial 0 failed with parameters: {'feature_fraction': 0.5} because of the following error: DTypePromotionError("The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float32DType'>)

,datetime,client,close,price_am,price_pm,price,y_lag_365,price_am_lag_1,price_am_lag_7,price_am_lag_365,price_pm_lag_1,price_pm_lag_7,price_pm_lag_365,price_am_moving_avg_7,price_am_moving_avg_30,price_pm_moving_avg_7,price_pm_moving_avg_30
464,2012-04-12,0,0,0.0,0.0,0.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.857143,2.366667,0.714286,1.966667
1172,2014-03-21,0,0,3.0,3.0,3.0,71.0,2.0,1.0,5.0,2.0,1.0,4.0,2.428571,1.300000,1.857143,0.933333
1179,2014-03-28,0,0,4.0,4.0,4.0,83.0,5.0,3.0,4.0,4.0,3.0,4.0,4.428571,2.033333,4.000000,1.666667
1416,2014-11-20,0,0,0.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.533333,0.142857,0.366667
377,2012-01-16,0,0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,0.142857,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,2014-02-17,0,0,0.0,0.0,0.0,47.0,2.0,0.0,1.0,1.0,0.0,0.0,0.714286,0.600000,0.428571,0.366667
1660,2015-07-22,0,0,0.0,0.0,0.0,39.0,1.0,0.0,0.0,0.0,0.0,0.0,0.857143,0.733333,0.285714,0.366667
241,2011-09-02,0,0,1.0,1.0,1.0,NaN,1.0,0.0,NaN,1.0,0.0,NaN,0.714286,0.300000,0.714286,0.233333
271,2011-10-02,0,0,1.0,1.0,1.0,NaN,2.0,2.0,NaN,1.0,2.0,NaN,1.142857,0.766667,1.000000,0.633333


In [35]:
dtrain = lgb.Dataset(x_train, y_train)
dtest = lgb.Dataset(y_test, y_test)

params = {'metric' : 'rmse'}
model = lgb.train(params, dtrain)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float32DType'>)

In [31]:
df.dtypes

datetime                  datetime64[ns]
y                                  int64
client                             int64
close                              int64
price_am                         float64
price_pm                         float64
is_train                           int64
price                            float64
y_lag_365                        float64
price_am_lag_1                   float64
price_am_lag_7                   float64
price_am_lag_365                 float64
price_pm_lag_1                   float64
price_pm_lag_7                   float64
price_pm_lag_365                 float64
price_am_moving_avg_7            float64
price_am_moving_avg_30           float64
price_pm_moving_avg_7            float64
price_pm_moving_avg_30           float64
dtype: object